In [ ]:
import torch
import torch.nn as nn
# import pandas as pd
import json
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import numpy as np
import random
import re
from nltk.tokenize import word_tokenize
from collections import Counter

# from ...src.utils.wiki_tokens.wiki_cleaner import getMorphemeList
# import ../src/utiils/wiki_token/wiki_cleaner

In [ ]:
with open("../tokens/wiki.json", "r") as file:
    tokeniser = json.load(file)

In [ ]:
with open("text", "r") as file:
    wiki = file.read()

In [ ]:

# def getMorphemeList(text):
#   text = text.lower()
#   text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
#   morpheme_list = word_tokenize(text)
  
#   # Count occurrences of each word
#   word_counts = Counter(morpheme_list)
  
#   # Filter out words that only appear once
#   morpheme_list = [word for word in morpheme_list if word_counts[word] > 1]
  
#   return morpheme_list


# words = getMorphemeList(wiki)

In [9]:
words = wiki.split(sep=" ")
window = 2
print (words[0])
words.pop(0)

def get_words(words,idx,window=2):
    sent = words[max(0,idx-window):min(idx+window+1,len(words))]
    if len(sent) > 1:
        rand_idx = random.randint(0,len(sent)-1)
        target = sent[rand_idx]
        del sent[rand_idx]

        return sent, target
        

In [ ]:
class embed_train_dataset(Dataset):
    def __init__(self, words, tokenizer):
        self.data = words
        self.tokenizer = tokenizer
    
    def __len__(self):
        return len(self.data)
    